In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding

import warnings
warnings.filterwarnings("ignore")

In [11]:
from scripts.utils import calculate_length, preprocess_article

ModuleNotFoundError: No module named 'joblib'

In [9]:
data = load_dataset("csv", data_files="../data/data/Amharic_corpus_merged_2023-04-16.csv")

NameError: name 'load_dataset' is not defined

In [ ]:
data

DatasetDict({
    train: Dataset({
        features: ['article', 'category'],
        num_rows: 61915
    })
})

In [ ]:
data['train'] = data['train'].map(calculate_length, batched=False)
data['train'] = data['train'].map(preprocess_article, batched=False)

Map:   0%|          | 0/61915 [00:00<?, ? examples/s]

Map:   0%|          | 0/61915 [00:00<?, ? examples/s]

In [ ]:
raw_datasets = data['train'].train_test_split(train_size=0.8, seed=42)

In [ ]:
category = list(set(data['train']['category']))

checkpoint = "rasyosef/bert-small-amharic"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Tokenize the dataset
category_to_id = {cat: idx for idx, cat in enumerate(category)}

def tokenize_function(example):
    inputs = tokenizer(example['article'], padding=True, truncation=True, max_length=512)
    inputs["labels"] = category_to_id[example["category"]]  # Assuming category is already integer-encoded
    return inputs



In [ ]:
tokenized_datasets = raw_datasets.map(tokenize_function)
#tokenized_datasets = tokenized_datasets.map(lambda examples: {'labels': examples['category']}, batched=True)
#tokenized_datasets = tokenized_datasets.remove_columns(['article', 'category'])

# Use a data collator to apply dynamic batches

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='pt')

tokenized_datasets.set_format("torch")

print(tokenized_datasets)

2024-05-30 14:21:33.434274: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 14:21:33.434374: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 14:21:33.589992: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


tokenizer_config.json:   0%|          | 0.00/1.23k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/223k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/608k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Map:   0%|          | 0/49532 [00:00<?, ? examples/s]

Map:   0%|          | 0/12383 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['article', 'category', 'word_count', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 49532
    })
    test: Dataset({
        features: ['article', 'category', 'word_count', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 12383
    })
})


In [ ]:
# Load the model

from transformers import AutoModelForSequenceClassification

# roberta-base

model = AutoModelForSequenceClassification.from_pretrained(
    checkpoint,
    num_labels=len(category),
    id2label = {i: lbl for i, lbl in enumerate(category)},
    label2id = {lbl: i for i, lbl in enumerate(category)},
    device_map="cuda"
)

config.json:   0%|          | 0.00/648 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/103M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at rasyosef/bert-small-amharic and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from transformers import TrainingArguments

batch_size = 64
epochs = 5

training_args = TrainingArguments(
    output_dir=checkpoint+"-finetuned",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=epochs,
    weight_decay=0.1,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=True,
    seed=42,
)


In [ ]:
import evaluate
import numpy as np

def compute_metrics(eval_preds):
  metric1 = evaluate.load("accuracy")
  metric2 = evaluate.load("precision")
  metric3 = evaluate.load("recall")
  metric4 = evaluate.load("f1")

  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)

  accuracy = metric1.compute(predictions=predictions, references=labels)["accuracy"]
  precision = metric2.compute(predictions=predictions, references=labels, average='weighted')["precision"]
  recall = metric3.compute(predictions=predictions, references=labels, average='weighted')["recall"]
  f1 = metric4.compute(predictions=predictions, references=labels, average='weighted')["f1"]

  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1": f1
  }

compute_metrics(([[1,0], [0,1]], [0,1]))

{'accuracy': 1.0, 'precision': 1.0, 'recall': 1.0, 'f1': 1.0}

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.257800,0.355941,0.872809,0.874915,0.872809,0.872637
2,0.202900,0.366543,0.879835,0.882212,0.879835,0.879827
3,0.168900,0.387127,0.877897,0.877604,0.877897,0.877458
4,0.149400,0.393662,0.878785,0.878166,0.878785,0.878390
5,0.139400,0.403864,0.877816,0.877487,0.877816,0.877440


/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning

TrainOutput(global_step=3870, training_loss=0.1836711873687823, metrics={'train_runtime': 1991.1542, 'train_samples_per_second': 124.38, 'train_steps_per_second': 1.944, 'total_flos': 9796823106416640.0, 'train_loss': 0.1836711873687823, 'epoch': 5.0})

In [ ]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Load metrics and evaluate the model

from torch.utils.data import DataLoader
eval_dataset = tokenized_datasets["test"].remove_columns([
    'article', 'category', 'word_count'
    ]).with_format("torch")



eval_dataloader = DataLoader(
    eval_dataset,
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

y_test, y_pred = [], []
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to('cuda') for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    y_pred.extend(predictions.cpu().numpy())
    y_test.extend(batch["labels"].cpu().numpy())


In [ ]:
import evaluate
import numpy as np

def compute_metrics(y_pred, y_test):
  metric1 = evaluate.load("accuracy")
  metric2 = evaluate.load("precision")
  metric3 = evaluate.load("recall")
  metric4 = evaluate.load("f1")

  #logits, labels = y_preds
  #predictions = np.argmax(logits, axis=-1)

  accuracy = metric1.compute(predictions=y_pred, references=y_test)["accuracy"]
  precision = metric2.compute(predictions=y_pred, references=y_test, average='weighted')["precision"]
  recall = metric3.compute(predictions=y_pred, references=y_test, average='weighted')["recall"]
  f1 = metric4.compute(predictions=y_pred, references=y_test, average='weighted')["f1"]

  return {
      "accuracy": accuracy,
      "precision": precision,
      "recall": recall,
      "f1": f1
  }


In [ ]:
compute_metrics(y_pred, y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.8761204877654849,
 'precision': 0.7220551122788942,
 'recall': 0.7356290230304802,
 'f1': 0.7285139172493781}

In [ ]:
compute_metrics(y_pred, y_test)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'accuracy': 0.8798352580150206,
 'precision': 0.8822123910985895,
 'recall': 0.8798352580150206,
 'f1': 0.8798268986044279}

In [ ]:
metric = evaluate.load("f1")
metric.compute(predictions=y_pred, references=y_test, average='weighted')

{'f1': 0.8757126049284634}

In [ ]:
import pandas as pd
df = pd.read_csv('/kaggle/input/amharic-news-corpus-merged/Amharic_corpus_merged_2023-04-16.csv')

In [ ]:
df.columns

Index(['article', 'category'], dtype='object')

In [ ]:
df['category'].value_counts(normalize='true')

category
Local News            0.333909
Politics              0.233885
Sports                0.168150
Business              0.148106
International News    0.105677
Entertainment         0.010256
Others                0.000016
Name: proportion, dtype: float64